In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-k66g47tm
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-k66g47tm
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=033651bbca5a6b8d72726ffc28a35857d7a4e75378beb2ab36820ec74a9306a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ik2udrzb/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


Aby była możliwość skompilowania programów poniżej należy w oknie Google Colab w zakładce "Środowisko wykonawcze" zamienić "none" na "GPU" a następnie wykonać powyższe 3 linijki kodu by zainstalować środowisko


CPU:

In [4]:
%%cu
#include <stdio.h>
#define DIM 1024
#define rnd( x ) (x * rand() / RAND_MAX)
#define INF 2e10f

struct Sphere {
    float   r,b,g;
    float   radius;
    float   x,y,z;
    float hit( float ox, float oy, float *n ) {
      float dx = ox - x;
      float dy = oy - y;
      if (dx*dx + dy*dy < radius*radius) {
        float dz = sqrtf( radius*radius - dx*dx - dy*dy );
        *n = dz / sqrtf( radius * radius );
        return dz + z;
      }
      return -INF;
    }
};

#define SPHERES 20

void kernel( Sphere *s, unsigned char *ptr, int x , int y ) {
    // map from threadIdx/BlockIdx to pixel position
    int offset = x + y * DIM;
    float   ox = (x - DIM/2);
    float   oy = (y - DIM/2);
  
    float   r=0, g=0, b=0;
    float   maxz = -INF;
    for(int i=0; i<SPHERES; i++) {
        float   n;
        float   t = s[i].hit( ox, oy, &n );
        if (t > maxz) {
          float fscale = n;
          r = s[i].r * fscale;
          g = s[i].g * fscale;
          b = s[i].b * fscale;
          maxz = t;
        }
    } 

    ptr[offset*4 + 0] = (int)(r * 255);
    ptr[offset*4 + 1] = (int)(g * 255);
    ptr[offset*4 + 2] = (int)(b * 255);
    ptr[offset*4 + 3] = 255;
}

struct DataBlock{
    unsigned char *dev_bitmap;
};

void save_to_file(unsigned char *ptr){
  FILE *fp=fopen("CPU_reyTraced.ppm","w");
  fprintf(fp,"P3\n%d %d\n255\n", DIM, DIM);
  for(int y=0;y<DIM;y++){
    for(int x=0;x<DIM;x++){
      int offset=x+y*DIM;
      fprintf(fp,"\n%d %d %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2]);
    }
  }
  fclose(fp);
}


int main(void) {
  
  clock_t start, end;
  start = clock();
  
  DataBlock data;
  unsigned char *bitmap = (unsigned char*)malloc(DIM* DIM*4* sizeof(unsigned char));
  int image_size = DIM* DIM*4;
  Sphere *s;

  Sphere *temp_s = (Sphere*)malloc(sizeof(Sphere)* SPHERES );

  for (int i=0; i<SPHERES; i++) {
    temp_s[i].r = rnd( 1.0f);
    temp_s[i].g= rnd( 1.0f);
    temp_s[1].b= rnd( 1.0f);
    temp_s[i].x =rnd( 1000.0f)- 500;
    temp_s[i].y =rnd( 1000.0f) - 500;
    temp_s[1].z = rnd( 1000.0f) - 500;
    temp_s[i].radius = rnd( 100.0f) + 20;
  }

  s = temp_s;
  
  for (int i=0; i<DIM; i++) {
    for (int j=0; j<DIM; j++) {
      kernel(s, bitmap, i, j);
    }
  }

  save_to_file(bitmap);
  
  end = clock();
  double elapsed_time = (double)(end - start) / CLOCKS_PER_SEC;
  printf("Compilation time: %f seconds\n", elapsed_time);

}

Compilation time: 0.336729 seconds



Aby wyświetlić efekt programu należy po lewej stronie okna google colab kliknąć na ikonę foldera następnie na pliku "CPU_reyTraced.ppm" w menu pod trzema kropkami kliknąć "Pobierz". Pobrany plik w formacie .ppm można przekonwertować w format .jpg / .png itp za pomocą dostępnych converterów Online by zobaczyć obraz Drugim sposobem jest otworzenie pliku za pomocą programu o nazwie IrfanVieww

GPU:

In [5]:
%%cu
#include <stdio.h>
#define DIM 1024
#define rnd( x ) (x * rand() / RAND_MAX)
#define INF 2e10f

struct Sphere {
    float   r,b,g;
    float   radius;
    float   x,y,z;
    __device__ float hit( float ox, float oy, float *n ) {
      float dx = ox - x;
      float dy = oy - y;
      if (dx*dx + dy*dy < radius*radius) {
        float dz = sqrtf( radius*radius - dx*dx - dy*dy );
        *n = dz / sqrtf( radius * radius );
        return dz + z;
      }
      return -INF;
    }
};

#define SPHERES 20

__global__ void kernel( Sphere *s, unsigned char *ptr ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;
    float   ox = (x - DIM/2);
    float   oy = (y - DIM/2);
  
    float   r=0, g=0, b=0;
    float   maxz = -INF;
    for(int i=0; i<SPHERES; i++) {
        float   n;
        float   t = s[i].hit( ox, oy, &n );
        if (t > maxz) {
          float fscale = n;
          r = s[i].r * fscale;
          g = s[i].g * fscale;
          b = s[i].b * fscale;
          maxz = t;
        }
    } 

    ptr[offset*4 + 0] = (int)(r * 255);
    ptr[offset*4 + 1] = (int)(g * 255);
    ptr[offset*4 + 2] = (int)(b * 255);
    ptr[offset*4 + 3] = 255;
}

struct DataBlock{
    unsigned char *dev_bitmap;
};

void save_to_file(unsigned char *data){
  FILE *fp=fopen("GPU_reyTraced.ppm","w");
  fprintf(fp,"P3\n%d %d\n255\n", DIM, DIM);
  for(int y=0;y<DIM;y++){
    for(int x=0;x<DIM;x++){
      int offset=x+y*DIM;
      fprintf(fp,"\n%d %d %d", data[offset*4+0], data[offset*4+1], data[offset*4+2]);
    }
  }
  fclose(fp);
}

int main(void) {
  
  cudaEvent_t start, stop;
  cudaEventCreate( &start );
  cudaEventCreate( &stop);
  cudaEventRecord( start, 0);
  
  DataBlock data;
  unsigned char *bitmap = (unsigned char*)malloc(DIM* DIM*4* sizeof(unsigned char));
  int image_size = DIM* DIM*4;
  unsigned char *dev_bitmap;
  cudaMalloc((void**)&dev_bitmap, image_size); 
  data.dev_bitmap=dev_bitmap;
  Sphere *s;

  cudaMalloc((void**)&s, sizeof(Sphere)* SPHERES );

  Sphere *temp_s = (Sphere*)malloc(sizeof(Sphere)* SPHERES );

  for (int i=0; i<SPHERES; i++) {
    temp_s[i].r = rnd( 1.0f);
    temp_s[i].g= rnd( 1.0f);
    temp_s[1].b= rnd( 1.0f);
    temp_s[i].x =rnd( 1000.0f)- 500;
    temp_s[i].y =rnd( 1000.0f) - 500;
    temp_s[1].z = rnd( 1000.0f) - 500;
    temp_s[i].radius = rnd( 100.0f) + 20;
  }

cudaMemcpy( s, temp_s, sizeof(Sphere)* SPHERES, cudaMemcpyHostToDevice);
free( temp_s);

dim3 grids(DIM/16,DIM/16);
dim3 threads(16,16);

kernel<<<grids,threads>>>(s, dev_bitmap);

cudaMemcpy( bitmap, dev_bitmap, image_size, cudaMemcpyDeviceToHost);

cudaEventRecord( stop, 0);
cudaEventSynchronize(stop);
float elapsedTime;  
cudaEventElapsedTime(&elapsedTime, start, stop);
printf("Compilation time: %f seconds\n", elapsedTime/1000);

cudaEventDestroy(start);
cudaEventDestroy(stop);


save_to_file(bitmap);
}

Compilation time: 0.002103 seconds



Aby wyświetlić efekt programu należy po lewej stronie okna google colab kliknąć na ikonę foldera następnie na pliku "GPU_reyTraced.ppm" w menu pod trzema kropkami kliknąć "Pobierz". Pobrany plik w formacie .ppm można przekonwertować w format .jpg / .png itp za pomocą dostępnych converterów Online by zobaczyć obraz Drugim sposobem jest otworzenie pliku za pomocą programu o nazwie IrfanVieww

Średni zysk w czasie obliczeń między CPU a GPU wynosi 200 tu krotnie mniejszy czas obliczeń na danym komputerze (0.336729 seconds => 0.002103 seconds)